This is the Colab notebook for the implementation of Object Proposals and Calculating Intersection of Unions (IoU). This code run for general classes and labels and not pertaining to our dataset. 

###### *This code is from chatGPT*

## Step 1: Install the necessary libraries such as PyTorch, Torchvision, and Detectron2.

First, we import the necessary libraries such as PyTorch, Torchvision, and Detectron2.



In [ ]:
!pip install torch torchvision detectron2
!pip install git+https://github.com/facebookresearch/detectron2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2 to /tmp/pip-req-build-oy4xy85b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2 /tmp/pip-req-build-oy4xy85b
  Resolved https://github.com/facebookresearch/detectron2 to commit 670b54e376d468958834bce380672d5a23d49922
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 19.7 MB/s eta 0:00:00
   

## Step 2: Import the required libraries
Next, we load the pre-trained Mask R-CNN model for instance segmentation from the Detectron2 model zoo. We use the "get_cfg" function to obtain the configuration for the model specified by its name, merge the default configuration with the model configuration, set a threshold for the model's prediction scores, and load the pre-trained weights. We then create a predictor object using this configuration.

In [ ]:
import torch
import torchvision
import numpy as np
import cv2
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer,VisImage
from detectron2.structures import BoxMode


## Step 3: Load the pre-trained model using Detectron2
We load an input image using the "cv2.imread" function, which returns a numpy array representation of the image.

In [ ]:
model = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
predictor = DefaultPredictor(cfg)


model_final_f10217.pkl: 178MB [00:02, 86.0MB/s]                          


## Step 4: Load the image and make a prediction using the pre-trained model

We pass this input image through the Mask R-CNN model using the predictor object we created earlier. This gives us the model's predictions for the input image, which includes the predicted bounding boxes and class labels for the objects in the image. You can change the image path as per your needs.

In [ ]:
im = cv2.imread("/content/operating-room.jpg")
outputs = predictor(im)


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## Step 5: Extract the bounding box coordinates and class labels from the prediction

We extract the predicted bounding box coordinates and class labels from the model's predictions using the "pred_boxes" and "pred_classes" attributes of the "outputs["instances"]" object returned by the predictor.

In [ ]:
pred_classes = outputs["instances"].pred_classes.tolist()
pred_boxes = outputs["instances"].pred_boxes.tensor.tolist()


In [ ]:
print(outputs)

{'instances': Instances(num_instances=3, image_height=650, image_width=1000, fields=[pred_boxes: Boxes(tensor([[ 463.2617,  231.3715,  519.5033,  278.9182],
        [ 295.2306,  412.6859,  382.3875,  508.3141],
        [ 924.3395,  341.4202, 1000.0000,  581.0680]], device='cuda:0')), scores: tensor([0.9174, 0.7018, 0.5236], device='cuda:0'), pred_classes: tensor([62, 56, 68], device='cuda:0'), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,

## Step 6: Create the scene graph by mapping objects to their relationships

We create a list of objects by mapping the predicted bounding boxes to their corresponding class labels, and a list of relationships between these objects based on their overlap. We calculate the overlap between two objects by computing their intersection-over-union (IoU) score, and consider them to be related if their IoU score is above a certain threshold (in this case, 0.5).

In [ ]:
def calculate_iou(box1, box2):
    '''Compute the intersection over union of two set of boxes, each box is [x1,y1,x2,y2].
    Args:
      box1: (tensor) bounding boxes, sized (4,).
      box2: (tensor) bounding boxes, sized (4,).
    Return:
      (tensor) iou.
    '''
    lt = np.zeros(2)
    rb = np.zeros(2)    # get inter-area left_top/right_bottom
    for i in range(2):
        if box1[i] > box2[i]:
            lt[i] = box1[i]
        else:
            lt[i] = box2[i]
        if box1[i+2] < box2[i+2]:
            rb[i] = box1[i+2]
        else:
            rb[i] = box2[i+2]
    wh = rb-lt
    wh[wh<0] = 0    # if no overlapping 
    inter = wh[0] * wh[1]
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])  
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    iou = inter / (area1 + area2 - inter)          
    return iou

In [ ]:
objects = []
relationships = []

# Create objects from predicted boxes
for i, box in enumerate(pred_boxes):
    obj = {}
    obj["class"] = pred_classes[i]
    obj["box"] = box
    objects.append(obj)

# Create relationships between objects based on overlap
for i, obj1 in enumerate(objects):
    for j, obj2 in enumerate(objects):
        if i == j:
            continue
        box1 = obj1["box"]
        box2 = obj2["box"]
        iou = calculate_iou(box1, box2)
        if iou > 0.5:
            relationships.append((i, j))


## Step 7: Visualize the scene graph using the detected bounding boxes and relationships

Finally, we visualize the scene graph by drawing the predicted bounding boxes for the objects and the relationships between them using the "Visualizer" class from Detectron2. We create a "Visualizer" object, which takes the input image, the dataset metadata (in this case, the COCO dataset metadata), and a scale factor as arguments. We then draw the predicted bounding boxes for each object using the "draw_box" method of the "Visualizer" object, and draw the relationships between objects using the "draw_line" method. Finally, we get the output image using the "get_image" method of the "Visualizer" object, and save it to a file using the "cv2.imwrite" function.


In [ ]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
for obj in objects:
    box = BoxMode.convert(obj["box"], BoxMode.XYXY_ABS, BoxMode.XYWH_ABS)
    label = obj["class"]
    v.draw_box(box, edge_color="red")
    v.draw_text(label, box[:2])
    print(label)
for i, j in relationships:
    v.draw_line(objects[i]["box"][:2], objects[j]["box"][:2], edge_color="blue")
output_image = v.output.get_image()[:, :, ::-1]
cv2.imwrite("/content/output_image2.jpg", output_image)


62
56
68


True

In [ ]:
print(objects)

[{'class': 62, 'box': [463.26171875, 231.37152099609375, 519.5032958984375, 278.918212890625]}, {'class': 56, 'box': [295.2305908203125, 412.6859130859375, 382.38751220703125, 508.31414794921875]}, {'class': 68, 'box': [924.3395385742188, 341.42022705078125, 1000.0, 581.0679931640625]}]


## FASTER RCNN AND NEURAL MOTIFS

This is an attempt at using [neural-motifs](https://github.com/rowanz/neural-motifs) to generate scene graphs. But apparently, neural motifs is not a python project therefore does not have a ```setup.py``` to build the repo. That's why [maskrcnn_benchmark](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch) is used.

In [ ]:
!git clone https://github.com/rowanz/neural-motifs.git


Cloning into 'neural-motifs'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 546.08 KiB | 8.40 MiB/s, done.
Resolving deltas: 100% (92/92), done.


After downloading the repository, you need to add it to your Python path. You can do that by running the following command in the neural-motifs directory:

In [ ]:
#!export PYTHONPATH=~/content/neural-motifs
#!export PYTHONPATH:'$/env/python:/home/rowan/code/scene-graph'
! echo $PYTHONPATH
%env PYTHONPATH=/env/python/home/rowan/code/scene-graph
! echo $PYTHONPATH

"/env/python/home/rowan/code/scene-graph"
env: PYTHONPATH=/env/python/home/rowan/code/scene-graph
/env/python/home/rowan/code/scene-graph


In [ ]:
import sys
sys.path.append('/content/neural-motifs')

In [ ]:
!echo $PYTHONPATH

/env/python/home/rowan/code/scene-graph


In [ ]:
%cd /content/neural-motifs/
#%cd /content/
!pwd

/content/neural-motifs
/content/neural-motifs


In [ ]:
!pip install Cython


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Attempt to run ```Makefile``` as mentioned in the [neural-motifs](https://github.com/rowanz/neural-motifs) repo.

In [ ]:
!apt-get install build-essential
!make -f Makefile

In [ ]:
!pip install -e /content/neural-motifs/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/neural-motifs
ERROR: file:///content/neural-motifs does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


Hence, I fail to use neural-motifs as the relation detection model. Moving on to maskrcnn_benchmark.